In [1]:
import os, sys, json
sys.path.append(os.path.realpath(f"{os.getcwd()}/../../"))

from autocog.pm.sta.backend import Backend
from autocog.pm.sta.syntax import Syntax
from autocog.pm.sta.automaton import Automaton as STA
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
# source = """
# format bool {
#     is enum("true","false");
# }
# prompt main {
#     is {
#         A is bool;
#         B[0:2] is {
#             C is bool;
#             D[2:4] is bool;
#         }
#         E[0:2] is {
#           F is bool;
#         }
#         G is {
#           H is bool;
#           I is bool;
#         }
#         J is bool;
#     }
#     channel {
#         to .B.C from ?inC;
#         to .E   from ?inE;
#         to .G   from ?inG;
#     }
# }"""
# samples = [
#     (
#       {},
#       {
#         "inC" : [ 'true', 'false' ],
#         "inE" : [ { 'F' : 'true' } ],
#         "inG" : { 'H' : 'true' }
#       }
#     ),
#     (
#       {},
#       {
#         "inC" : [ 'true' ],
#         "inE" : [],
#         "inG" : { 'H' : 'true' }
#       }
#     )
# ]

source = """
prompt main {
    is {
        topic is text<20>;
        question is text<50>;
        choices[4] is {
            value is text<40>;
            correct is enum("yes","no");
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        _ as "You are answering a multiple choice questionnaire.";
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}"""
samples = [
    (
        {},
        {
            "topic" : "the topic of the question",
            "question" : "an exmaple question to show how it compiles",
            "choices" : [
                "the first choice",
                "the 2nd choice",
                "yet another 3rd choice",
                "final 4th choice"
            ]
        }
    )
]

# source = """
# prompt main {
#     is {
#         choices[4] is text<40>;
#         thought[1:3] is text<20>;
#         answer is repeat(.choices);
#     }
#     channel {
#         to .choices from ?choices;
#     }
#     return {
#         from .answer;
#     }
#     annotate {
#         _ as "You are answering a multiple choice questionnaire.";
#         .choices as "possible choices";
#         .thought as "think about the choices";
#         .answer  as "repeat the correct choice";
#     }
# }"""
# samples = [
#     (
#         {},
#         {
#             "choices" : [
#                 "first",
#                 "second",
#                 "third",
#                 "fourth"
#             ]
#         }
#     )
# ]

In [3]:
from autocog import CogArch
from autocog.architecture.utility import PromptTee
from autocog.lm import Llama

arch = CogArch(pipe=PromptTee(prefix='qna', tee=sys.stdout))
llama = Llama.create(model_path="/data/models/llama-2-7b-chat.Q4_K_M.gguf", n_ctx=4096)
arch.orchestrator.LMs.update({ 'text' : Llama(**llama, completion_kwargs={ 'max_tokens' : 20, 'temperature' : 0.4 }) })

In [4]:
syntax = Syntax(header_pre_post=('[INST]\n<<SYS>>You are an expert interacting with the world using an interactive prompting system.<</SYS>>','[/INST]'))
backend = Backend(source=source)

sta = STA(prompt=backend.program.prompts['main'])
sta.build_abstract()
sta.build_concrete()

for sample in samples:
    fta = sta.instantiate(syntax, stacks=sample[0], inputs=sample[1])
    fta.simplify()
    ftt = fta.greedy(lm=arch.orchestrator.LMs['text'])

In [5]:
display(wrap_graphviz(sta.toGraphViz_abstract()))
display(wrap_graphviz(sta.toGraphViz_concrete()))
display(wrap_graphviz(fta.toGraphViz()))
display(wrap_graphviz(ftt.toGraphViz()))